<img align="right" src="images/tf-small.png" width="128"/>
<img align="right" src="images/etcbc.png"/>
<img align="right" src="images/dans-small.png"/>

# Search Introduction

*Search* in Text-Fabric is a template based way of looking for structural patterns in your dataset.

It is inspired by the idea of
[topographic query](http://books.google.nl/books?id=9ggOBRz1dO4C),
as worked out in 
[MQL](https://github.com/ETCBC/shebanq/wiki/Documents/MQL-Query-Guide.pdf)
which has been implemented in 
[Emdros](http://emdros.org).
See also [pitfalls of MQL](https://etcbc.github.io/bhsa/mql#pitfalls-of-mql)

Within Text-Fabric we have the unique possibility to combine the ease of formulating search templates for
complicated syntactical patterns with the power of programmatically processing the results.

This notebook will show you how to get up and running.

See the notebook
[searchFromMQL](searchFromMQL.ipynb)
for examples how MQL queries can be expressed in Text-Fabric search.

## Alternative for hand-coding

Search is a powerful feature for a wide range of purposes.

Quite a bit of the implementation work has been dedicated to optimize performance.
Yet I do not pretend to have found optimal strategies for all 
possible search templates.
Some search tasks may turn out to be somewhat costly or even very costly.

That being said, I think search might turn out helpful in many cases,
especially by reducing the amount of hand-coding needed to work with special subsets of your data.

## Easy command

Search is as simple as saying (just an example)

```python
results = A.search(template)
A.show(results)
```

See all ins and outs in the
[search template docs](https://dans-labs.github.io/text-fabric/Use/Search/#search-templates).

In [1]:
%load_ext autoreload
%autoreload 2

# Incantation

The ins and outs of installing Text-Fabric, getting the corpus, and initializing a notebook are
explained in the [start tutorial](start.ipynb).

In [2]:
from tf.app import use

In [3]:
A = use('bhsa', hoist=globals())

Using etcbc/bhsa/tf - c r1.5 in /Users/dirk/text-fabric-data
Using etcbc/phono/tf - c r1.2 in /Users/dirk/text-fabric-data
Using etcbc/parallels/tf - c r1.2 in /Users/dirk/text-fabric-data


**Documentation:** <a target="_blank" href="https://etcbc.github.io/bhsa" title="provenance of BHSA = Biblia Hebraica Stuttgartensia Amstelodamensis">BHSA</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Writing/Hebrew" title="('Hebrew characters and transcriptions',)">Character table</a> <a target="_blank" href="https://etcbc.github.io/bhsa/features/hebrew/c/0_home.html" title="BHSA feature documentation">Feature docs</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Apps/Bhsa/" title="bhsa API documentation">bhsa API</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/General/" title="text-fabric-api">Text-Fabric API 7.1.1</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Use/Search/" title="Search Templates Introduction and Reference">Search Reference</a>

# Basic search command

We start with the most simple form of issuing a query.
Let's look for the proper nouns in 1 Samuel.
We also want to show the clauses in which they occur.

All work involved in searching takes place under the hood.

In [4]:
query = '''
book book=Samuel_I
  clause
    word sp=nmpr
'''
results = A.search(query)
A.table(results, end=10)

  0.47s 1868 results


n | p | book | clause | word
--- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:1">1_Samuel 1:1</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel" sec="1_Samuel">1_Samuel</a></span>|<span class="hb"><span  class="hlup" >וַ</span><span  class="hlup" >יְהִי֩ </span><span  class="hlup" >אִ֨ישׁ </span><span  class="hlup" >אֶחָ֜ד </span><span  class="hlup" >מִן־</span><span  class="hlup" >הָ</span><span  class="hlup" >רָמָתַ֛יִם </span><span  class="hlup" >צֹופִ֖ים </span><span  class="hlup" >מֵ</span><span  class="hlup" >הַ֣ר </span><span  class="hl hlup" >אֶפְרָ֑יִם </span></span>|<span class="hb"><span  class="hl hlup" >אֶפְרָ֑יִם </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:1">1_Samuel 1:1</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel" sec="1_Samuel">1_Samuel</a></span>|<span class="hb"><span  class="hlup" >וּ</span><span  class="hlup" >שְׁמֹ֡ו </span><span  class="hl hlup" >אֶ֠לְקָנָה </span><span  class="hlup" >בֶּן־</span><span  class="hlup" >יְרֹחָ֧ם </span><span  class="hlup" >בֶּן־</span><span  class="hlup" >אֱלִיה֛וּא </span><span  class="hlup" >בֶּן־</span><span  class="hlup" >תֹּ֥חוּ </span><span  class="hlup" >בֶן־</span><span  class="hlup" >צ֖וּף </span><span  class="hlup" >אֶפְרָתִֽי׃ </span></span>|<span class="hb"><span  class="hl hlup" >אֶ֠לְקָנָה </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:1">1_Samuel 1:1</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel" sec="1_Samuel">1_Samuel</a></span>|<span class="hb"><span  class="hlup" >וּ</span><span  class="hlup" >שְׁמֹ֡ו </span><span  class="hlup" >אֶ֠לְקָנָה </span><span  class="hlup" >בֶּן־</span><span  class="hl hlup" >יְרֹחָ֧ם </span><span  class="hlup" >בֶּן־</span><span  class="hlup" >אֱלִיה֛וּא </span><span  class="hlup" >בֶּן־</span><span  class="hlup" >תֹּ֥חוּ </span><span  class="hlup" >בֶן־</span><span  class="hlup" >צ֖וּף </span><span  class="hlup" >אֶפְרָתִֽי׃ </span></span>|<span class="hb"><span  class="hl hlup" >יְרֹחָ֧ם </span></span>
4| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:1">1_Samuel 1:1</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel" sec="1_Samuel">1_Samuel</a></span>|<span class="hb"><span  class="hlup" >וּ</span><span  class="hlup" >שְׁמֹ֡ו </span><span  class="hlup" >אֶ֠לְקָנָה </span><span  class="hlup" >בֶּן־</span><span  class="hlup" >יְרֹחָ֧ם </span><span  class="hlup" >בֶּן־</span><span  class="hl hlup" >אֱלִיה֛וּא </span><span  class="hlup" >בֶּן־</span><span  class="hlup" >תֹּ֥חוּ </span><span  class="hlup" >בֶן־</span><span  class="hlup" >צ֖וּף </span><span  class="hlup" >אֶפְרָתִֽי׃ </span></span>|<span class="hb"><span  class="hl hlup" >אֱלִיה֛וּא </span></span>
5| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:1">1_Samuel 1:1</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel" sec="1_Samuel">1_Samuel</a></span>|<span class="hb"><span  class="hlup" >וּ</span><span  class="hlup" >שְׁמֹ֡ו </span><span  class="hlup" >אֶ֠לְקָנָה </span><span  class="hlup" >בֶּן־</span><span  class="hlup" >יְרֹחָ֧ם </span><span  class="hlup" >בֶּן־</span><span  class="hlup" >אֱלִיה֛וּא </span><span  class="hlup" >בֶּן־</span><span  class="hl hlup" >תֹּ֥חוּ </span><span  class="hlup" >בֶן־</span><span  class="hlup" >צ֖וּף </span><span  class="hlup" >אֶפְרָתִֽי׃ </span></span>|<span class="hb"><span  class="hl hlup" >תֹּ֥חוּ </span></span>
6| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:1">1_Samuel 1:1</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel" sec="1_Samuel">1_Samuel</a></span>|<span class="hb"><span  class="hlup" >וּ</span><span  class="hlup" >שְׁמֹ֡ו </span><span  class="hlup" >אֶ֠לְקָנָה </span><span  class="hlup" >בֶּן־</span><span  class="hlup" >יְרֹחָ֧ם </span><span  class="hlup" >בֶּן־</span><span  class="hlup" >אֱלִיה֛וּא </span><span  class="hlup" >בֶּן־</span><span  class="hlup" >תֹּ֥חוּ </span><span  class="hlup" >בֶן־</span><span  class="hl hlup" >צ֖וּף </span><span  class="hlup" >אֶפְרָתִֽי׃ </span></span>|<span class="hb"><span  class="hl hlup" >צ֖וּף </span></span>
7| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=2&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:2">1_Samuel 1:2</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel" sec="1_Samuel">1_Samuel</a></span>|<span class="hb"><span  class="hlup" >שֵׁ֤ם </span><span  class="hlup" >אַחַת֙ </span><span  class="hl hlup" >חַנָּ֔ה </span></span>|<span class="hb"><span  class="hl hlup" >חַנָּ֔ה </span></span>
8| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=2&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:2">1_Samuel 1:2</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel" sec="1_Samuel">1_Samuel</a></span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >שֵׁ֥ם </span><span  class="hlup" >הַ</span><span  class="hlup" >שֵּׁנִ֖ית </span><span  class="hl hlup" >פְּנִנָּ֑ה </span></span>|<span class="hb"><span  class="hl hlup" >פְּנִנָּ֑ה </span></span>
9| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=2&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:2">1_Samuel 1:2</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel" sec="1_Samuel">1_Samuel</a></span>|<span class="hb"><span  class="hlup" >לִ</span><span  class="hl hlup" >פְנִנָּה֙ </span><span  class="hlup" >יְלָדִ֔ים </span></span>|<span class="hb"><span  class="hl hlup" >פְנִנָּה֙ </span></span>
10| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=2&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:2">1_Samuel 1:2</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel" sec="1_Samuel">1_Samuel</a></span>|<span class="hb"><span  class="hlup" >וּ</span><span  class="hlup" >לְ</span><span  class="hl hlup" >חַנָּ֖ה </span><span  class="hlup" >אֵ֥ין </span><span  class="hlup" >יְלָדִֽים׃ </span></span>|<span class="hb"><span  class="hl hlup" >חַנָּ֖ה </span></span>

We can show them in phonetic representation as well:

In [5]:
A.table(results, end=10, fmt='text-phono-full')

n | p | book | clause | word
--- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:1">1_Samuel 1:1</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel" sec="1_Samuel">1_Samuel</a></span>|<span class="trb"><span  class="hlup" >wa</span><span  class="hlup" >yᵊhˌî </span><span  class="hlup" >ʔˌîš </span><span  class="hlup" >ʔeḥˈāḏ </span><span  class="hlup" >min-</span><span  class="hlup" >hā</span><span  class="hlup" >rāmāṯˈayim </span><span  class="hlup" >ṣôfˌîm </span><span  class="hlup" >mē</span><span  class="hlup" >hˈar </span><span  class="hl hlup" >ʔefrˈāyim </span></span>|<span class="trb"><span  class="hl hlup" >ʔefrˈāyim </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:1">1_Samuel 1:1</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel" sec="1_Samuel">1_Samuel</a></span>|<span class="trb"><span  class="hlup" >û</span><span  class="hlup" >šᵊmˈô </span><span  class="hl hlup" >ʔelqānˌā </span><span  class="hlup" >ben-</span><span  class="hlup" >yᵊrōḥˈām </span><span  class="hlup" >ben-</span><span  class="hlup" >ʔᵉlîhˈû </span><span  class="hlup" >ben-</span><span  class="hlup" >tˌōḥû </span><span  class="hlup" >ven-</span><span  class="hlup" >ṣˌûf </span><span  class="hlup" >ʔefrāṯˈî . </span></span>|<span class="trb"><span  class="hl hlup" >ʔelqānˌā </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:1">1_Samuel 1:1</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel" sec="1_Samuel">1_Samuel</a></span>|<span class="trb"><span  class="hlup" >û</span><span  class="hlup" >šᵊmˈô </span><span  class="hlup" >ʔelqānˌā </span><span  class="hlup" >ben-</span><span  class="hl hlup" >yᵊrōḥˈām </span><span  class="hlup" >ben-</span><span  class="hlup" >ʔᵉlîhˈû </span><span  class="hlup" >ben-</span><span  class="hlup" >tˌōḥû </span><span  class="hlup" >ven-</span><span  class="hlup" >ṣˌûf </span><span  class="hlup" >ʔefrāṯˈî . </span></span>|<span class="trb"><span  class="hl hlup" >yᵊrōḥˈām </span></span>
4| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:1">1_Samuel 1:1</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel" sec="1_Samuel">1_Samuel</a></span>|<span class="trb"><span  class="hlup" >û</span><span  class="hlup" >šᵊmˈô </span><span  class="hlup" >ʔelqānˌā </span><span  class="hlup" >ben-</span><span  class="hlup" >yᵊrōḥˈām </span><span  class="hlup" >ben-</span><span  class="hl hlup" >ʔᵉlîhˈû </span><span  class="hlup" >ben-</span><span  class="hlup" >tˌōḥû </span><span  class="hlup" >ven-</span><span  class="hlup" >ṣˌûf </span><span  class="hlup" >ʔefrāṯˈî . </span></span>|<span class="trb"><span  class="hl hlup" >ʔᵉlîhˈû </span></span>
5| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:1">1_Samuel 1:1</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel" sec="1_Samuel">1_Samuel</a></span>|<span class="trb"><span  class="hlup" >û</span><span  class="hlup" >šᵊmˈô </span><span  class="hlup" >ʔelqānˌā </span><span  class="hlup" >ben-</span><span  class="hlup" >yᵊrōḥˈām </span><span  class="hlup" >ben-</span><span  class="hlup" >ʔᵉlîhˈû </span><span  class="hlup" >ben-</span><span  class="hl hlup" >tˌōḥû </span><span  class="hlup" >ven-</span><span  class="hlup" >ṣˌûf </span><span  class="hlup" >ʔefrāṯˈî . </span></span>|<span class="trb"><span  class="hl hlup" >tˌōḥû </span></span>
6| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:1">1_Samuel 1:1</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel" sec="1_Samuel">1_Samuel</a></span>|<span class="trb"><span  class="hlup" >û</span><span  class="hlup" >šᵊmˈô </span><span  class="hlup" >ʔelqānˌā </span><span  class="hlup" >ben-</span><span  class="hlup" >yᵊrōḥˈām </span><span  class="hlup" >ben-</span><span  class="hlup" >ʔᵉlîhˈû </span><span  class="hlup" >ben-</span><span  class="hlup" >tˌōḥû </span><span  class="hlup" >ven-</span><span  class="hl hlup" >ṣˌûf </span><span  class="hlup" >ʔefrāṯˈî . </span></span>|<span class="trb"><span  class="hl hlup" >ṣˌûf </span></span>
7| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=2&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:2">1_Samuel 1:2</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel" sec="1_Samuel">1_Samuel</a></span>|<span class="trb"><span  class="hlup" >šˈēm </span><span  class="hlup" >ʔaḥˌaṯ </span><span  class="hl hlup" >ḥannˈā </span></span>|<span class="trb"><span  class="hl hlup" >ḥannˈā </span></span>
8| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=2&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:2">1_Samuel 1:2</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel" sec="1_Samuel">1_Samuel</a></span>|<span class="trb"><span  class="hlup" >wᵊ</span><span  class="hlup" >šˌēm </span><span  class="hlup" >ha</span><span  class="hlup" >ššēnˌîṯ </span><span  class="hl hlup" >pᵊninnˈā </span></span>|<span class="trb"><span  class="hl hlup" >pᵊninnˈā </span></span>
9| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=2&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:2">1_Samuel 1:2</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel" sec="1_Samuel">1_Samuel</a></span>|<span class="trb"><span  class="hlup" >li</span><span  class="hl hlup" >fᵊninnˌā </span><span  class="hlup" >yᵊlāḏˈîm </span></span>|<span class="trb"><span  class="hl hlup" >fᵊninnˌā </span></span>
10| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=2&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:2">1_Samuel 1:2</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel" sec="1_Samuel">1_Samuel</a></span>|<span class="trb"><span  class="hlup" >û</span><span  class="hlup" >lᵊ</span><span  class="hl hlup" >ḥannˌā </span><span  class="hlup" >ʔˌên </span><span  class="hlup" >yᵊlāḏˈîm . </span></span>|<span class="trb"><span  class="hl hlup" >ḥannˌā </span></span>

The hyperlinks take us all to the beginning of the book of 1 Samuel, but it is more convenient to jump to the
precise verse in which the result occurs. It is better to put hyperlinks
underneath the words instead. That is column 3.

Note in passing that we can choose start and/or end points in the results list.

In [6]:
A.table(results, start=8, end=13, linked=3)

n | p | book | clause | word
--- | --- | --- | --- | ---
8| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=2&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:2">1_Samuel 1:2</a>|<span class="trb">1_Samuel</span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >שֵׁ֥ם </span><span  class="hlup" >הַ</span><span  class="hlup" >שֵּׁנִ֖ית </span><span  class="hl hlup" >פְּנִנָּ֑ה </span></span>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=2&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel 1:2" sec="1_Samuel 1:2"><span  class="hl hlup" >פְּנִנָּ֑ה </span></a></span>
9| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=2&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:2">1_Samuel 1:2</a>|<span class="trb">1_Samuel</span>|<span class="hb"><span  class="hlup" >לִ</span><span  class="hl hlup" >פְנִנָּה֙ </span><span  class="hlup" >יְלָדִ֔ים </span></span>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=2&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel 1:2" sec="1_Samuel 1:2"><span  class="hl hlup" >פְנִנָּה֙ </span></a></span>
10| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=2&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:2">1_Samuel 1:2</a>|<span class="trb">1_Samuel</span>|<span class="hb"><span  class="hlup" >וּ</span><span  class="hlup" >לְ</span><span  class="hl hlup" >חַנָּ֖ה </span><span  class="hlup" >אֵ֥ין </span><span  class="hlup" >יְלָדִֽים׃ </span></span>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=2&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel 1:2" sec="1_Samuel 1:2"><span  class="hl hlup" >חַנָּ֖ה </span></a></span>
11| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=3&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:3">1_Samuel 1:3</a>|<span class="trb">1_Samuel</span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >לִ</span><span  class="hlup" >זְבֹּ֛חַ </span><span  class="hlup" >לַ</span><span  class="hl hlup" >יהוָ֥ה </span><span  class="hlup" >צְבָאֹ֖ות </span><span  class="hlup" >בְּ</span><span  class="hlup" >שִׁלֹ֑ה </span></span>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=3&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel 1:3" sec="1_Samuel 1:3"><span  class="hl hlup" >יהוָ֥ה </span></a></span>
12| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=3&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:3">1_Samuel 1:3</a>|<span class="trb">1_Samuel</span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >לִ</span><span  class="hlup" >זְבֹּ֛חַ </span><span  class="hlup" >לַ</span><span  class="hlup" >יהוָ֥ה </span><span  class="hlup" >צְבָאֹ֖ות </span><span  class="hlup" >בְּ</span><span  class="hl hlup" >שִׁלֹ֑ה </span></span>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=3&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel 1:3" sec="1_Samuel 1:3"><span  class="hl hlup" >שִׁלֹ֑ה </span></a></span>
13| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=3&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="1_Samuel 1:3">1_Samuel 1:3</a>|<span class="trb">1_Samuel</span>|<span class="hb"><span  class="hlup" >וְ</span><span  class="hlup" >שָׁ֞ם </span><span  class="hlup" >שְׁנֵ֣י </span><span  class="hlup" >בְנֵֽי־</span><span  class="hl hlup" >עֵלִ֗י </span><span  class="hlup" >חָפְנִי֙ </span><span  class="hlup" >וּ</span><span  class="hlup" >פִ֣נְחָ֔ס </span><span  class="hlup" >כֹּהֲנִ֖ים </span><span  class="hlup" >לַ</span><span  class="hlup" >יהוָֽה׃ </span></span>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Samuel_I&chapter=1&verse=3&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="1_Samuel 1:3" sec="1_Samuel 1:3"><span  class="hl hlup" >עֵלִ֗י </span></a></span>

We can show the results more fully with `show()`.

In [7]:
A.show(results, start=1, end=3)



**book** *1*





**book** *2*





**book** *3*



Or one verse in phonetic representation.

In [8]:
A.show(results, start=1, end=1, fmt='text-phono-full')



**book** *1*



# Condense results

There are two fundamentally different ways of presenting the results: condensed and uncondensed.

In **uncondensed** view, all results are listed individually.
You can keep track of which parts belong to which results.
The display can become unwieldy.

This is the default view, because it is the straightest, most logical, answer to your query.

In **condensed** view all nodes of all results are grouped in containers first (e.g. verses), and then presented 
container by container.
You loose the information of what parts belong to what result.

Here is an example of the difference.

In [9]:
query = '''
book book=Genesis
  chapter chapter=1
    verse verse=1
      sentence
% order is not important!
        word nu=sg
        word nu=pl
'''

Note that you can have comments in a search template. Comment lines start with a `%`.

In [10]:
results = A.search(query)
A.table(results, linked=4, withPassage=True)

  0.95s 6 results


n | p | book | chapter | verse | sentence | word | word
--- | --- | --- | --- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:1">Genesis 1:1</a>|<span class="trb">Genesis</span>|<span class="trb">Genesis 1</span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 1:1</span></span><span  class="hlup" >בְּ</span><span  class="hl hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hl hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1" sec="Genesis 1:1"><span  class="hlup" >בְּ</span><span  class="hl hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hl hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></a></span>|<span class="hb"><span  class="hl hlup" >רֵאשִׁ֖ית </span></span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֑ים </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:1">Genesis 1:1</a>|<span class="trb">Genesis</span>|<span class="trb">Genesis 1</span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 1:1</span></span><span  class="hlup" >בְּ</span><span  class="hl hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hl hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1" sec="Genesis 1:1"><span  class="hlup" >בְּ</span><span  class="hl hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hl hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></a></span>|<span class="hb"><span  class="hl hlup" >רֵאשִׁ֖ית </span></span>|<span class="hb"><span  class="hl hlup" >שָּׁמַ֖יִם </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:1">Genesis 1:1</a>|<span class="trb">Genesis</span>|<span class="trb">Genesis 1</span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 1:1</span></span><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hl hlup" >בָּרָ֣א </span><span  class="hl hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1" sec="Genesis 1:1"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hl hlup" >בָּרָ֣א </span><span  class="hl hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></a></span>|<span class="hb"><span  class="hl hlup" >בָּרָ֣א </span></span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֑ים </span></span>
4| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:1">Genesis 1:1</a>|<span class="trb">Genesis</span>|<span class="trb">Genesis 1</span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 1:1</span></span><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hl hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hl hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1" sec="Genesis 1:1"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hl hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hl hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></a></span>|<span class="hb"><span  class="hl hlup" >בָּרָ֣א </span></span>|<span class="hb"><span  class="hl hlup" >שָּׁמַ֖יִם </span></span>
5| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:1">Genesis 1:1</a>|<span class="trb">Genesis</span>|<span class="trb">Genesis 1</span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 1:1</span></span><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hl hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hl hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1" sec="Genesis 1:1"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hl hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hl hlup" >אָֽרֶץ׃ </span></a></span>|<span class="hb"><span  class="hl hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֑ים </span></span>
6| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:1">Genesis 1:1</a>|<span class="trb">Genesis</span>|<span class="trb">Genesis 1</span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 1:1</span></span><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hl hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hl hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1" sec="Genesis 1:1"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hl hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hl hlup" >אָֽרֶץ׃ </span></a></span>|<span class="hb"><span  class="hl hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hl hlup" >שָּׁמַ֖יִם </span></span>

There are two plural and three singular words in Genesis 1:1.
Search templates do not specify order, so all six combinations qualify as results.

Let's expand the results display:

In [11]:
A.show(results)



**book** *1*





**book** *2*





**book** *3*





**book** *4*





**book** *5*





**book** *6*



As you see, the results are listed per result tuple, even if they occur all in the same verse.
This way you can keep track of what exactly belongs to each result.

Now in condensed mode:

In [12]:
A.show(results, condensed=True)



**verse** *1*



Here we have all words of all results in one display. But we cannot see that each results has two words, let alone which ones.

# Custom highlighting

Note that we can apply different highlight colors to different parts of the result.
The words in the pair are member 5 and 6 of the result tuples.
The members that we do not map, will not be highlighted.
The members that we map to the empty string will be highlighted with the default color.

**NB:** Choose your colors from the
[CSS specification](https://developer.mozilla.org/en-US/docs/Web/CSS/color_value).

In [14]:
A.show(results, condensed=False, colorMap={4: '', 5: 'cyan', 6: 'magenta'})



**book** *1*





**book** *2*





**book** *3*





**book** *4*





**book** *5*





**book** *6*



Color mapping works best for uncondensed results. If you condense results, some nodes may occupy
different positions in different results. It is unpredictable which color will be used 
for such nodes:

In [15]:
A.show(results, condensed=True, colorMap={4: '', 5: 'cyan', 6: 'magenta'})



**verse** *1*



You can specify to what container you want to condense. By default, everything is condensed to verses.

Let's change that to phrases:

In [16]:
A.show(results, condensed=True, condenseType='phrase', colorMap={4: '', 5: 'cyan', 6: 'magenta'})



**phrase** *1*





**phrase** *2*





**phrase** *3*





**phrase** *4*



# Constraining order
You can stipulate an order on the words in your template.
You only have to put a relational operator between them.
Say we want only results where the plural follows the singular.

In [17]:
query = '''
book book=Genesis
  chapter chapter=1
    verse verse=1
      sentence
        word nu=sg
        < word nu=pl
'''

In [18]:
results = A.search(query)
A.table(results)

  0.93s 4 results


n | p | book | chapter | verse | sentence | word | word
--- | --- | --- | --- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1">Genesis 1</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis" sec="Genesis">Genesis</a></span>|<span class="trb">Genesis 1</span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 1:1</span></span><span  class="hlup" >בְּ</span><span  class="hl hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hl hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >בְּ</span><span  class="hl hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hl hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hl hlup" >רֵאשִׁ֖ית </span></span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֑ים </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1">Genesis 1</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis" sec="Genesis">Genesis</a></span>|<span class="trb">Genesis 1</span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 1:1</span></span><span  class="hlup" >בְּ</span><span  class="hl hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hl hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >בְּ</span><span  class="hl hlup" >רֵאשִׁ֖ית </span><span  class="hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hl hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hl hlup" >רֵאשִׁ֖ית </span></span>|<span class="hb"><span  class="hl hlup" >שָּׁמַ֖יִם </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1">Genesis 1</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis" sec="Genesis">Genesis</a></span>|<span class="trb">Genesis 1</span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 1:1</span></span><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hl hlup" >בָּרָ֣א </span><span  class="hl hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hl hlup" >בָּרָ֣א </span><span  class="hl hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hl hlup" >בָּרָ֣א </span></span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֑ים </span></span>
4| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1">Genesis 1</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis" sec="Genesis">Genesis</a></span>|<span class="trb">Genesis 1</span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 1:1</span></span><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hl hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hl hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hl hlup" >בָּרָ֣א </span><span  class="hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hl hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hl hlup" >בָּרָ֣א </span></span>|<span class="hb"><span  class="hl hlup" >שָּׁמַ֖יִם </span></span>

We can also require the words to be adjacent.

In [19]:
query = '''
book book=Genesis
  chapter chapter=1
    verse verse=1
      sentence
        word nu=sg
        <: word nu=pl
'''

In [20]:
results = A.search(query)
A.table(results)
A.show(results, condensed=False, colorMap={5: 'lightsalmon', 6: 'mediumaquamarine'})

  1.00s 1 result


n | p | book | chapter | verse | sentence | word | word
--- | --- | --- | --- | --- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1">Genesis 1</a>|<span class="trb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis" sec="Genesis">Genesis</a></span>|<span class="trb">Genesis 1</span>|<span class="hb"><span class="vn"><span  class="hlup" >Genesis 1:1</span></span><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hl hlup" >בָּרָ֣א </span><span  class="hl hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hlup" >בְּ</span><span  class="hlup" >רֵאשִׁ֖ית </span><span  class="hl hlup" >בָּרָ֣א </span><span  class="hl hlup" >אֱלֹהִ֑ים </span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הַ</span><span  class="hlup" >שָּׁמַ֖יִם </span><span  class="hlup" >וְ</span><span  class="hlup" >אֵ֥ת </span><span  class="hlup" >הָ</span><span  class="hlup" >אָֽרֶץ׃ </span></span>|<span class="hb"><span  class="hl hlup" >בָּרָ֣א </span></span>|<span class="hb"><span  class="hl hlup" >אֱלֹהִ֑ים </span></span>



**book** *1*



# Custom feature display

We would like to see the gender, number and person for words.
The way to do that, is to perform a `A.prettySetup(features)` first.

In [21]:
A.displaySetup(extraFeatures='ps gn nu')

In [22]:
A.show(results, condensed=False, colorMap={5: 'lightsalmon', 6: 'mediumaquamarine'})



**book** *1*



The features without meaningful values have been left out. We can also change that by passing a set of values
we think are not meaningful. The default set is 

```python
{None, 'NA', 'none', 'unknown'}
```

In [23]:
A.displaySetup(noneValues=set())
A.show(results, condensed=False, colorMap={5: 'lightsalmon', 6: 'mediumaquamarine'})



**book** *1*



This makes clear that it is convenient to keep `None` in the `noneValues`:

In [24]:
A.displaySetup(noneValues={None})
A.show(results, condensed=False, colorMap={5: 'lightsalmon', 6: 'mediumaquamarine'})



**book** *1*



We can even choose to suppress other values, e.g. the male gender values and the singular number values.

In [25]:
A.displaySetup(noneValues={None, 'NA', 'unknown', 'none', 'm', 'sg'})
A.show(results, condensed=False, colorMap={5: 'lightsalmon', 6: 'mediumaquamarine'})



**book** *1*



In the rest of the notebook we stick to our normal setup, so we reset the extra features.

In [26]:
A.displayReset()
A.show(results, condensed=False, colorMap={5: 'lightsalmon', 6: 'mediumaquamarine'})



**book** *1*



# Show your own tuples

So far we have `show()`n the results of searches.
But you can also construct your own tuples and show them.

Whereas you can use search to get a pretty good approximation of what you want, most of the times
you do not arrive precisely at your destination.

Here is an example where we use search to come close, and then work our way to produce the end result.

## Disagreement in number

We look for clauses with a one-word subject that does not agree in number with its predicate.

In our search templates we cannot formulate that a feature has different values on two nodes in the template.
We could spell out all possible combinations of values and make a search template for each of them, 
but that is needlessly complex.

Let's first use search to find all clauses containing a one word subject and a predicate.
And, to narrow down it further, we require that the word in the subject and the verb in the predicate are
marked for number.

(You may want to consult the feature docs, see the link at the start of the notebook, where `Bhsa()` is called).

Note that the order of the phrases does not matter.

In [27]:
query = '''
clause
    phrase function=Subj
        =: word nu=sg|pl
        :=
    phrase function=Pred|PreO
        word sp=verb
             nu=sg|pl
'''
results = A.search(query)

  1.90s 10638 results


Now the hand coding begins. We are going to extract the tuples we want.

In [28]:
wantedResults = tuple(
    (subj, pred)
    for (clause, phraseS, subj, phraseV, pred) in results
    if F.nu.v(subj) != F.nu.v(pred)
)
print(f'{len(wantedResults)} filtered results')

469 filtered results


And now we can show them:

In [30]:
A.table(wantedResults, start=1, end=3, colorMap={1: 'lightsalmon', 2: 'mediumaquamarine'})

n | p | word | word
--- | --- | --- | ---
1| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:1">Genesis 1:1</a>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=1&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:1" sec="Genesis 1:1"><span  class="hl"  style="background-color: lightsalmon;" >אֱלֹהִ֑ים </span></a></span>|<span class="hb"><span  class="hl"  style="background-color: mediumaquamarine;" >בָּרָ֣א </span></span>
2| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:3">Genesis 1:3</a>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=3&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:3" sec="Genesis 1:3"><span  class="hl"  style="background-color: lightsalmon;" >אֱלֹהִ֖ים </span></a></span>|<span class="hb"><span  class="hl"  style="background-color: mediumaquamarine;" >יֹּ֥אמֶר </span></span>
3| <a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="show this passage in SHEBANQ" sec="Genesis 1:4">Genesis 1:4</a>|<span class="hb"><a href="https://shebanq.ancient-data.org/hebrew/text?book=Genesis&chapter=1&verse=4&version=c&mr=m&qw=q&tp=txt_p&tr=hb&wget=v&qget=v&nget=vt" title="Genesis 1:4" sec="Genesis 1:4"><span  class="hl"  style="background-color: lightsalmon;" >אֱלֹהִ֛ים </span></a></span>|<span class="hb"><span  class="hl"  style="background-color: mediumaquamarine;" >יַּ֧רְא </span></span>

In [31]:
A.show(wantedResults, start=1, end=3, colorMap={1: 'lightsalmon', 2: 'mediumaquamarine'})



**word** *1*





**word** *2*





**word** *3*



Now suppose that we want to highlight the non-qal verb forms with a different color.

We have to assing colors to the members of our tuples:

In [32]:
highlights = {}
for (subj, pred) in wantedResults:
    highlights[subj] = 'lightsalmon'
    highlights[pred] = 'mediumaquamarine' if F.vs.v(pred) == 'qal' else 'yellow'

Now we can call show with the `highlights` parameter instead of the `colorMap` parameter.

In [33]:
A.show(wantedResults, start=2, end=3, highlights=highlights)



**word** *2*





**word** *3*



As you see, you have total control.

# Next

You know how to run queries and show off with their results.

The next thing is to dive deeper into the power of templates:
[advanced](searchAdvanced.ipynb)

---

basic
[advanced](searchAdvanced.ipynb)
[sets](searchSets.ipynb)
[relations](searchRelations.ipynb)
[quantifiers](searchQuantifiers.ipynb)
[rough](searchRough.ipynb)
[gaps](searchGaps.ipynb)

---

* **[display](display.ipynb)** become an expert in creating pretty displays of your text structures
* **[share](share.ipynb)** draw in other people's data and let them use yours
* **[export](export.ipynb)** export your dataset as an Emdros database

Back to [start](start.ipynb)